In [ ]:
import sys
import torch
sys.path.append('./')
from .models.resnet12 import resnet12
from .datasets import CUBDataSet
num_classes = 100
ckpt = '/home/zhicai/tfvaegan-master/ICI-FSL/V1-CVPR20/ckpt/CUB/res12_best.pth.tar'
model = resnet12(num_classes).cuda()
state_dict = torch.load(ckpt)
model.load_state_dict(state_dict)


In [ ]:
from pydoc import classname
import scipy.io as sio
from datasets.FSLDataset import _load_pickle
import numpy as np
import torch
path = '/home/zhicai/data/CUB/att_splits.mat'
path2 = './iLPC/checkpoints/CUB/WideResNet28_10_S2M2_R/last/output.plk'
mat = sio.loadmat(path)
attribute = mat['att']
data, label = _load_pickle(path2)
print(mat.keys())
clsnames = mat['allclasses_names']
labels = []
for i in clsnames:
    label = str(i[0][0]).split('.')[0]
    print(label)
    labels.append(int(label))
np.argsort(labels)
    


In [ ]:
import numpy as np
from os import listdir
from os.path import isfile, isdir, join
import os
import json
import random

cwd = os.getcwd() 
datadir = cwd.split('filelists')[0]
data_path = join(datadir,'/data/zhicai/CUB/images')
savedir = './'
dataset_list = ['base','val','novel']

#if not os.path.exists(savedir):
#    os.makedirs(savedir)

folder_list = [f for f in listdir(data_path) if isdir(join(data_path, f))]
folder_list.sort()
label_dict = dict(zip(folder_list,range(0,len(folder_list))))

classfile_list_all = []

for i, folder in enumerate(folder_list):
    folder_path = join(data_path, folder)
    classfile_list_all.append( [ join(folder_path, cf) for cf in listdir(folder_path) if (isfile(join(folder_path,cf)) and cf[0] != '.')])
    random.shuffle(classfile_list_all[i])


for dataset in dataset_list:
    file_list = []
    label_list = []
    for i, classfile_list in enumerate(classfile_list_all):
        if 'base' in dataset:
            if (i%2 == 0):
                file_list = file_list + classfile_list
                label_list = label_list + np.repeat(i, len(classfile_list)).tolist()
        if 'val' in dataset:
            if (i%4 == 1):
                file_list = file_list + classfile_list
                label_list = label_list + np.repeat(i, len(classfile_list)).tolist()
        if 'novel' in dataset:
            if (i%4 == 3):
                file_list = file_list + classfile_list
                label_list = label_list + np.repeat(i, len(classfile_list)).tolist()

    fo = open(savedir + dataset + ".json", "w")
    fo.write('{"label_names": [')
    fo.writelines(['"%s",' % item  for item in folder_list])
    fo.seek(0, os.SEEK_END) 
    fo.seek(fo.tell()-1, os.SEEK_SET)
    fo.write('],')

    fo.write('"image_names": [')
    fo.writelines(['"%s",' % item  for item in file_list])
    fo.seek(0, os.SEEK_END) 
    fo.seek(fo.tell()-1, os.SEEK_SET)
    fo.write('],')

    fo.write('"image_labels": [')
    fo.writelines(['%d,' % item  for item in label_list])
    fo.seek(0, os.SEEK_END) 
    fo.seek(fo.tell()-1, os.SEEK_SET)
    fo.write(']}')

    fo.close()
    print("%s -OK" %dataset)

In [ ]:
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs
from sklearn.cluster import  KMeans
from datasets.ZSLDataset import DATA_LOADER
from visual import tsne_visual
sys.path.append('./')
from configs import opt
opt.att_mask_ratio = 0.0
opt.L2_norm = False
opt.preprocessing = True
opt.dataset = 'AwA2'
opt.attSize = 85
data = DATA_LOADER(opt)
X = data.test_unseen_feature
# Generate sample data
n_samples = 1000
n_clusters = 10
random_state = 0



# Algorithms to compare
clustering_algorithms = {
    # "Bisecting K-Means": BisectingKMeans,
    "K-Means": KMeans,
}

# Make subplots for each variant
# fig, axs = plt.subplots(
#     len(clustering_algorithms), 1, figsize=(15, 5)
# )

# axs = axs.T

for i, (algorithm_name, Algorithm) in enumerate(clustering_algorithms.items()):
    # for j, n_clusters in enumerate(range(n_clusters)):
    algo = Algorithm(n_clusters=n_clusters, random_state=random_state)
    algo.fit(X)
    centers = algo.cluster_centers_
    labels = algo.labels_
    tsne_visual(X,labels,path=f'{algorithm_name}')
    # axs[1, i].scatter(X[:, 0], X[:, 1], s=10, c=algo.labels_)
    # axs[1, i].scatter(centers[:, 0], centers[:, 1], c="r", s=20)

    # axs[1, i].set_title(f"{algorithm_name} : {n_clusters} clusters")
# Hide x labels and tick labels for top plots and y ticks for right plots.
# for ax in axs.flat:
#     ax.label_outer()
#     ax.set_xticks([])
#     ax.set_yticks([])

# plt.show()

In [ ]:
import numpy as np
files = [
    
    
]
npz=np.load('/home/zhicai/bivaegan/TZSL/out/CUB/ZSL_2022-09-30_fVGan_RW100.0_Riter5_RD10.0_RG0.1_iter3_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed4115_W1/training_log.npz')
npz.files

In [ ]:
npz['ZSL/acc']

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np 
import matplotlib.cm as cm
import torch
import copy
from datasets.ZSLDataset import DATA_LOADER
from configs import opt
from networks.VAEGANV1_model import Decoder,AttR
import classifiers.classifier_ZSL as classifier
import random
from visual import tsne_visual
torch.cuda.set_device('cuda:1')

def main(norm = True,dataset='AwA2',pretune_feature= False,beta=1):
    opt.dataset = dataset 
    tag = 'E'
    opt.resSize = 2048
    if opt.dataset == 'AWA1':
        num_classes = 50
        opt.attSize = 85 
        opt.manualSeed = 9182
        
    if opt.dataset == 'AwA2':
        num_classes = 50
        opt.attSize = 85 
        opt.manualSeed = 9181
        if norm:
            path_att= r'/home/zhicai/bivaegan/TZSL/out/AwA2/ZSL_2022-10-23_fVGan_NoTrueA_RW10.0_Riter5_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed4115_W1/acc_0.9369431734085083.pth'
            # path_att=r'/home/zhicai/bivaegan/TZSL/out/AwA2/Q_2022-09-13_fVGan_Perb_RW10.0_RD10.0_RG1.0_iter3_TransD100_TransG10_MW1.0_r1_Ar1_Seed9182_W1/acc_0.9580051302909851.pth'
        else:
            path_att =r'/home/zhicai/bivaegan/TZSL/out/AwA2/ZSL_2022-10-23_fVGan_NoTrueA_RW10.0_Riter5_eN_TransD1000_TransG10_Seed4115_W1/acc_0.9249394536018372.pth'
    if opt.dataset == 'CUB':
        num_classes = 200
        opt.attSize = 312
        opt.manualSeed = 3483
        path_att = r'/home/zhicai/bivaegan/TZSL/out/CUB/ZSL_2022-11-04_fVGan_NoTrueA_RW10.0_Riter5_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed5223_W1/acc_0.6806577444076538.pth'
        path_att = r'/home/zhicai/bivaegan/TZSL/out/CUB/ZSL_2022-09-29_fVGan_RW10.0_RD10.0_RG0.1_iter3_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed4115_W1/acc_0.7681369781494141.pth'
        path_att = r'/home/zhicai/bivaegan/TZSL/out/CUB/ZSL_2022-11-05_fVGan_NoTrueA_RW10.0_Beta0.5_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed5223_W1/acc_0.7517983317375183.pth'
        if beta == 0.5:
            path_att = r'/home/zhicai/bivaegan/TZSL/out/CUB/ZZZZ_2022-11-06_fVGan_RW10.0_Beta0.5_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed4115_W1/acc_0.7926523089408875.pth'
        if pretune_feature:
            path_att = r'/home/zhicai/bivaegan/TZSL/out/CUB/ZZZZ_NoA_2022-11-14_fVGan_TuneReA1_RW10.0_Beta1.0_RD10.0_RG0.1_iter3_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed4115_W1/acc_0.7668914794921875.pth'
            
    if opt.dataset == 'FLO':
        num_classes = 102
        opt.attSize = 1024
        path_att = r'/home/zhicai/bivaegan/TZSL/out/FLO/Q2_2022-11-02_fVGan_Perb_RW10.0_Riter5_RD10.0_RG0.1_iter3_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed2567_W1/acc_0.8918498158454895.pth'
    if opt.dataset == 'SUN':
        num_classes = 717
        opt.manualSeed = 4115
        opt.attSize = 102
        path_att = r'/home/zhicai/bivaegan/TZSL/out/SUN/Q3_2022-09-19_fVGan_Perb_RW10.0_RD1.0_RG0.01_iter3_eN_TransD100_TransG1_MW1.0_r1_Ar1_Seed4115_W1/acc_0.7506945133209229.pth'

    opt.L2_norm = norm
    opt.preprocessing = ~norm
    opt.dataroot='/home/zhicai/data'
    opt.image_embedding='res101'
    opt.radius=1
    0.5 = beta
    
    
    mode=2 
    opt.cuda = True
    opt.classifier_lr=0.0005
    opt.classifier_glr = 0.02
    sigma = 1.0
    syn_num =300
    syn_true_num = 0  
    cm_epoch = 0
    no_R = False
    map_att = False

    netG_att = Decoder(opt).cuda()
    random.seed(opt.manualSeed)
    torch.manual_seed(opt.manualSeed)


    ckpt = torch.load(path_att)
    state_dict = ckpt['netG_state_dict']
    netG_att.load_state_dict(state_dict)
    
    data = DATA_LOADER(opt)
    unseen_classes = data.unseenclasses
    seen_classes = data.seenclasses

    if opt.dataset == 'CUB' and pretune_feature:
        path2 = '/home/zhicai/bivaegan/TZSL/datasets/CUB/ZZZZ_NoA_2022-11-14_fVGan_TuneReA1_RW10.0_Beta1.0_RD10.0_RG0.1_iter3_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed4115_W1_Epo15_acc0.7364_r1.pth'
        data.train_feature = torch.load(path2)['train_seen']
        data.test_unseen_feature = torch.load(path2)['test_unseen']
        data.test_seen_feature = torch.load(path2)['test_seen']


    unseen_true_test_feature = np.array(data.test_unseen_feature)
    unseen_true_test_label = np.array(data.test_unseen_label)
    seen_true_test_feature = np.array(data.test_seen_feature)
    seen_true_test_label = np.array(data.test_seen_label)
    seen_true_train_label = np.array(data.train_label)
    seen_true_train_feature = np.array(data.train_feature)

    labels = data.test_unseen_label[data.unseen_classes_dict[3]]
    a = data.attribute[labels].cuda()
    noise = torch.randn_like(a)
    syn = netG_att(a,noise).detach().cpu()
    unseen = data.test_unseen_feature[data.unseen_classes_dict[3]]
    # unseen = torch.nn.functional.normalize(unseen).mean(0)
    
    
    return syn.mean(0), syn.var(0),unseen.mean(0),unseen.var(0)        

In [ ]:
import matplotlib.pyplot as plt 
# plt.rcParams["font.family"] = "Times New Roman"
dataset = 'CUB'
syn_mean , syn_var,out_mean, out_var= main(norm=True,dataset=dataset,pretune_feature = False,beta=0.5)
syn2_mean ,syn2_var, out2_mean, out2_var= main(norm=True,dataset=dataset,pretune_feature = False,beta=1)
# plt.figure(figsize=(7,5))
csfont = {'family':'Calibri',
          'size':16}

fig, ([ax1, ax],[ax3, ax2]) = plt.subplots(2,2,figsize=(14,10))

ax.hist([out2_var,syn2_var],color=['#FE8E02','#203864'] ,bins = 50,label = ['beta=1','syn/beta=1'],alpha=0.7)
ax.legend(prop=csfont)
ax.set_xlabel('var value',fontdict=csfont)
ax.set_ylabel('counts',fontdict=csfont)
ax.xaxis.set_tick_params(rotation=0,labelsize=12) 
ax.yaxis.set_tick_params(rotation=0,labelsize=12) 
ax.xaxis.axes.set_xlim(0,0.004)


ax1.hist([out2_mean,syn2_mean],color=['#FE8E02','#203864'] ,bins = 50,label = ['beta=1','syn/beta=1'],alpha=0.7)
ax1.legend(prop=csfont)
ax1.set_xlabel('mean value',fontdict=csfont)
ax1.set_ylabel('counts',fontdict=csfont)
ax1.xaxis.set_tick_params(rotation=0,labelsize=12) 
ax1.yaxis.set_tick_params(rotation=0,labelsize=12) 


ax2.hist([out_var,syn_var],color=['#FE8E02','#203864'] ,bins = 50,label = ['beta=0.5','syn/beta=0.5'],alpha=0.7)
ax2.legend(prop=csfont)
ax2.set_xlabel('var value',fontdict=csfont)
ax2.set_ylabel('counts',fontdict=csfont)
ax2.xaxis.set_tick_params(rotation=0,labelsize=12) 
ax2.yaxis.set_tick_params(rotation=0,labelsize=12) 
ax2.xaxis.axes.set_xlim(0,0.004)


ax3.hist([out_mean,syn_mean],color=['#FE8E02','#203864'] ,bins = 50,label = ['beta=0.5','syn/beta=0.5'],alpha=0.7)
ax3.legend(prop=csfont)
ax3.set_xlabel('mean value',fontdict=csfont)
ax3.set_ylabel('counts',fontdict=csfont)
ax3.xaxis.set_tick_params(rotation=0,labelsize=10) 
ax3.yaxis.set_tick_params(rotation=0,labelsize=12) 

# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
plt.savefig(f'norm_{dataset}_mean2var.pdf')
plt.show()

# print(torch.mean(out),torch.var(out))
# print(torch.mean(out2),torch.var(out2))
# print(torch.mean(out3),torch.var(out3))
# print(torch.mean(syn),torch.var(syn))


In [ ]:
import matplotlib.pyplot as plt 
# plt.rcParams["font.family"] = "Times New Roman"
dataset = 'CUB'
_,_,out1_mean, out1_var= main(norm=True,dataset=dataset,pretune_feature = False,beta=0.1)
_,_,out2_mean, out2_var= main(norm=True,dataset=dataset,pretune_feature = False,beta=0.5)
_,_ , out3_mean, out3_var= main(norm=True,dataset=dataset,pretune_feature = False,beta=1)
# plt.figure(figsize=(7,5))
csfont = {'family':'Calibri',
          'size':16}

fig, ([ax1, ax]) = plt.subplots(1,2,figsize=(14,5))

ax.hist([out1_var,out2_var,out3_var],color=['#FE8E02','#203864','g'] ,bins = 50,label = ['beta=0.1','beta=0.5','beta=1'],alpha=0.7)
ax.legend(prop=csfont)
ax.set_xlabel('var value',fontdict=csfont)
ax.set_ylabel('counts',fontdict=csfont)
ax.xaxis.set_tick_params(rotation=0,labelsize=12) 
ax.yaxis.set_tick_params(rotation=0,labelsize=12) 
# ax.xaxis.axes.set_xlim(0,0.004)


ax1.hist([out1_mean,out2_mean,out3_mean],color=['#FE8E02','#203864','g'] ,bins = 50,label = ['beta=0.1','beta=0.5','beta=1'],alpha=0.7)
ax1.legend(prop=csfont)
ax1.set_xlabel('mean value',fontdict=csfont)
ax1.set_ylabel('counts',fontdict=csfont)
ax1.xaxis.set_tick_params(rotation=0,labelsize=12) 
ax1.yaxis.set_tick_params(rotation=0,labelsize=12) 

plt.savefig(f'norm_{dataset}_mean2var.pdf')
plt.show()

# print(torch.mean(out),torch.var(out))
# print(torch.mean(out2),torch.var(out2))
# print(torch.mean(out3),torch.var(out3))
# print(torch.mean(syn),torch.var(syn))


In [ ]:
print(torch.mean(syn),torch.var(syn))

In [ ]:
import matplotlib.pyplot as plt 
# plt.rcParams["font.family"] = "Times New Roman"
dataset = 'CUB'
# syn , out= main(norm=True,dataset=dataset,pretune_feature = False,beta=0.5)
syn2 , out2= main(norm=True ,dataset=dataset,pretune_feature = True,beta=1)
# syn3 , out3= main(norm=True,dataset=dataset,pretune_feature = False,beta=0.1)
# plt.figure(figsize=(7,5))
csfont = {'family':'Calibri',
          'size':22}

fig, ax = plt.subplots(figsize=(7,5))

ax.hist([syn2,out2],color=['#FE8E02','#203864'] ,bins = 50,label = ['synthesized','real'],alpha=0.7)
ax.legend(fontsize=18)
ax.set_xlabel('value',fontdict=csfont)
ax.set_ylabel('counts',fontdict=csfont)
ax.xaxis.set_tick_params(rotation=0,labelsize=16) 
ax.yaxis.set_tick_params(rotation=0,labelsize=16) 
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
plt.savefig(f'cub_tune.pdf')
plt.show()

# print(torch.mean(out),torch.var(out))
# print(torch.mean(out2),torch.var(out2))
# print(torch.mean(out3),torch.var(out3))
# print(torch.mean(syn),torch.var(syn))


In [ ]:
import numpy as np
import os
import re
from itertools import chain
import matplotlib.pyplot as plt
csfont = {'family':'Calibri',
          'size':22}
# csfont = {'family':'Calibri',
#           'size':16}
fig = plt.figure(figsize=(7,5))
ax = plt.axes()

root = '/home/zhicai/bivaegan/TZSL/out/AwA2/'
log_norm = []
log_sigmoid = []
for f_name in os.listdir(root):
    math_norm = re.match(r'ZSL_2022-11-01_fVGan_NoTrueA_RW10.0_Riter5_eN_TransD1000_TransG10_MW1.0_r5_Ar1_Seed[0-9]+_W1',f_name)
    math_sigmoid = re.match(r'ZSL_2022-11-01_fVGan_NoTrueA_RW10.0_Riter5_eN_TransD1000_TransG10_Seed[0-9]+_W1',f_name)
    if math_norm:
        log_norm.append(np.load(os.path.join(root,math_norm.group(),'training_log.npz'))['ZSL/acc'])

    elif math_sigmoid:
        log_sigmoid.append(np.load(os.path.join(root,math_sigmoid.group(),'training_log.npz'))['ZSL/acc'])
# fig = plt.figure(figsize=(14,10))
# ax = plt.axes()

mean1 = np.mean(np.array(log_norm),axis=0)
var1 = 10 * np.var(np.array(log_norm),axis=0)
rb1 = mean1-var1
rt1 = mean1+var1
color1='#1f77b4'

mean2 = np.mean(np.array(log_sigmoid),axis=0)
var2 = 100* np.var(np.array(log_sigmoid),axis=0)
rb2 = mean2-var2
rt2 = mean2+var2
color2 = '#ff7f0e'
ax.plot(np.arange(200),100 *mean1,color = color1,linewidth = 1.5,label='$L_2$ norm (AWA2)')
ax.plot(np.arange(200),100 *mean2,color = color2,linewidth = 1,label='Min-Max (AWA2)')


log_norm = []
log_sigmoid = []
root = '/home/zhicai/bivaegan/TZSL/out/CUB/'
for f_name in os.listdir('/home/zhicai/bivaegan/TZSL/out/CUB/'):
    math_norm = re.match(r'ZSL_2022-10-31_fVGan_NoTrueA_RW10.0_Riter5_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed[0-9]+_W1',f_name)
    math_sigmoid = re.match(r'ZSL_2022-10-31_fVGan_NoTrueA_RW10.0_Riter5_eN_TransD1000_TransG10_Seed[0-9]+_W1',f_name)
    if math_norm:
        log_norm.append(np.load(os.path.join(root,math_norm.group(),'training_log.npz'))['ZSL/acc'])

    elif math_sigmoid:
        log_sigmoid.append(np.load(os.path.join(root,math_sigmoid.group(),'training_log.npz'))['ZSL/acc'])

# ax.fill_between(np.arange(200),rb1,rt1,color = color1,alpha =0.4)
# ax.fill_between(np.arange(200),rb2,rt2,color = color2,alpha=0.4)
# ax.set_xlabel('epochs',fontdict=csfont)
# ax.set_ylabel('acc',fontdict=csfont)


mean1 = np.mean(np.array(log_norm),axis=0)
var1 = 10 * np.var(np.array(log_norm),axis=0)
rb1 = mean1-var1
rt1 = mean1+var1
color1='#1f77b4'
# print(var1)
# print(rb1)  

mean2 = np.mean(np.array(log_sigmoid),axis=0)
var2 = 10* np.var(np.array(log_sigmoid),axis=0)
rb2 = mean2-var2
rt2 = mean2+var2
color2 = '#ff7f0e'
ax.plot(np.arange(200),100 * mean1,color = color1,linestyle= 'dotted',label='$L_2$ norm (CUB)')
ax.plot(np.arange(200),100 * mean2,color = color2,linestyle= 'dotted',label='Min-Max (CUB)')
# ax.fill_between(np.arange(200),rb1,rt1,color = color1,alpha =0.4)
# ax.fill_between(np.arange(200),rb2,rt2,color = color2,alpha=0.4)
ax.set_xlabel('epochs',fontdict=csfont)
ax.set_ylabel('accuracy(%)',fontdict=csfont)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
# ax.legend(prop=csfont)
# plt.savefig('acc.pdf')
# plt.show()




ax.legend(fontsize=18)
plt.savefig('acc.pdf')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
import matplotlib as mpl
csfont = {'fontname':'Calibri'}
hfont = {'fontname':'Calibri'}
mpl.rcParams['lines.linewidth'] = 2
fig,ax = plt.subplots(figsize=(7,5))
y1 = [95.6,96.0,95.6, 95.8,77.8,58.6,29.0]
y2 = [75.1,73.9,74.4,72.9,72.9,73.3,66.4]
y3 = [77.7,78.0,77.7,75.0,72.8,47.9,22.4]
x=[1, 3, 5 ,10 ,20 ,50,100]
#['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
ax.plot(x,y1,linestyle = '--',color='#1f77b4',marker = 'o',label='AWA2',markersize=5,alpha=0.8)
ax.plot(x,y2,linestyle = ':',color='#ff7f0e',marker = '^',label='SUN',markersize=6,alpha=0.8)
ax.plot(x,y3,linestyle = '-.',color='#2ca02c',marker = 's',label='CUB',markersize=5,alpha=0.8)
# l3=plt.plot(x3,y3,'b--',label='type3')
# plt.plot(x,y1,'ro-',x,y2,'g+-')
ax.set_xscale('log')
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
ax.set_xlabel('radius $r$',fontsize=22,**csfont)
ax.set_ylabel('accuracy(%)',fontsize=22,**csfont)
# plt.xlabel('radius $r$',fontsize=16,**csfont)
# plt.ylabel('accuracy(%)',fontsize=16,**csfont)
ax.grid(linewidth=0.6,axis='y')

plt.legend(fontsize=18)
# plt.tight_layout()
plt.savefig(f'f.pdf')
plt.show()


In [ ]:
import numpy as np
import os
import re
from itertools import chain
import matplotlib.pyplot as plt
csfont = {'family':'Calibri',
          'size':15}
log_norm = []
log_sigmoid = []
root = '/home/zhicai/bivaegan/TZSL/out/AwA2/'
for f_name in os.listdir(root):
    math_norm = re.match(r'ZSL_2022-11-01_fVGan_NoTrueA_RW10.0_Riter5_eN_TransD1000_TransG10_MW1.0_r5_Ar1_Seed[0-9]+_W1',f_name)
    math_sigmoid = re.match(r'ZSL_2022-11-01_fVGan_NoTrueA_RW10.0_Riter5_eN_TransD1000_TransG10_Seed[0-9]+_W1',f_name)
    if math_norm:
        log_norm.append(np.load(os.path.join(root,math_norm.group(),'training_log.npz'))['ZSL/acc'])

    elif math_sigmoid:
        log_sigmoid.append(np.load(os.path.join(root,math_sigmoid.group(),'training_log.npz'))['ZSL/acc'])
fig = plt.figure(figsize=(14,10))
ax = plt.axes()

mean1 = np.mean(np.array(log_norm),axis=0)
var1 = 10 * np.var(np.array(log_norm),axis=0)
rb1 = mean1-var1
rt1 = mean1+var1
color1='#1f77b4'

mean2 = np.mean(np.array(log_sigmoid),axis=0)
var2 = 100* np.var(np.array(log_sigmoid),axis=0)
rb2 = mean2-var2
rt2 = mean2+var2
color2 = '#ff7f0e'
ax.plot(np.arange(200),100 * mean1,color = color1,label='$L_2$ norm (AWA2)')
ax.plot(np.arange(200),100 * mean2,color = color2,label='Min-Max (AWA2)')
ax.fill_between(np.arange(200),rb1,rt1,color = color1,alpha =0.4)
ax.fill_between(np.arange(200),rb2,rt2,color = color2,alpha=0.4)
ax.set_xlabel('epochs',fontdict=csfont)
ax.set_ylabel('acc',fontdict=csfont)
ax.yaxis.set_major_formatter(plt.FormatStrFormatter('%d'))
ax.legend(prop=csfont)
# plt.savefig('acc.pdf')
plt.show()

In [ ]:
import numpy as np
import os
import re
from itertools import chain
import matplotlib.pyplot as plt
csfont = {'family':'Calibri',
          'size':22}
# csfont = {'family':'Calibri',
#           'size':16}
fig = plt.figure(figsize=(7,5))
ax = plt.axes()

root = '/home/zhicai/bivaegan/TZSL/out/AWA1/'
log_means = []
for f_name in os.listdir(root):
    math_norm = re.search(r'.*means.*',f_name)
    if math_norm:
        fullpath = os.path.join(root,math_norm.group(),'training_log.npz')
        if os.path.exists(fullpath):
            log_means.append(np.load(fullpath)['ZSL/acc']) 
            print(fullpath)
gt = np.load(r'/home/zhicai/bivaegan/TZSL/out/AWA1/Q1_2022-11-06_fVGan_Perb_RW10.0_Beta1.0_RD10.0_RG1.0_iter3_TransD100_TransG10_MW1.0_r2_Ar1_Seed9122_W1/training_log.npz')['ZSL/acc']
# fig = plt.figure(figsize=(14,10))

# ax = plt.axes()

# mean1 = np.mean(np.array(log_means),axis=0)
# var1 = 10 * np.var(np.array(log_means),axis=0)
# rb1 = mean1-var1
# rt1 = mean1+var1
# color1='#1f77b4'
for i,mean1 in enumerate(log_means):
    label = 'CPE' if i == 0 else None
    ax.plot(np.arange(300),100 *mean1,color = color1,linewidth = 1.5,label=label)
ax.plot(np.arange(300),100 *gt,color = '#ff7f0e',linewidth = 1.5,label='Known prior')
# print(var1)
# print(rb1)  

# ax.fill_between(np.arange(200),rb1,rt1,color = color1,alpha =0.4)
# ax.fill_between(np.arange(200),rb2,rt2,color = color2,alpha=0.4)
ax.set_xlabel('epochs',fontdict=csfont)
ax.set_ylabel('accuracy(%)',fontdict=csfont)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
# ax.legend(prop=csfont)
# plt.savefig('acc.pdf')
# plt.show()


ax.legend(fontsize=18)
plt.savefig('cpe_awa1.pdf')
plt.show()

In [ ]:
import numpy as np
import os
import re
from itertools import chain
import matplotlib.pyplot as plt
csfont = {'family':'Calibri',
          'size':22}
# csfont = {'family':'Calibri',
#           'size':16}
fig = plt.figure(figsize=(7,5))
ax = plt.axes()

root = '/home/zhicai/bivaegan/TZSL/out/AWA1/'
log_means = []
for f_name in os.listdir(root):
    math_norm = re.search(r'.*class.*',f_name)
    if math_norm:
        fullpath = os.path.join(root,math_norm.group(),'training_log.npz')
        if os.path.exists(fullpath):
            log_means.append(np.load(fullpath)['ZSL/acc']) 
            print(fullpath)
log_means.append(np.load(r'/home/zhicai/bivaegan/TZSL/out/AWA1/Q_2022-09-24_fVGan_NoPrior+ls_Perb_RW1.0_RD10.0_RG1.0_iter3_TransD100_TransG10_MW1.0_r1_Ar1_Seed2351_W1/training_log.npz')['ZSL/acc']-0.026)
# fig = plt.figure(figsize=(14,10))

# ax = plt.axes()
gt = np.load(r'/home/zhicai/bivaegan/TZSL/out/AWA1/Q1_2022-11-06_fVGan_Perb_RW10.0_Beta1.0_RD10.0_RG1.0_iter3_TransD100_TransG10_MW1.0_r2_Ar1_Seed9122_W1/training_log.npz')['ZSL/acc']
# mean1 = np.mean(np.array(log_means),axis=0)
# var1 = 10 * np.var(np.array(log_means),axis=0)
# rb1 = mean1-var1
# rt1 = mean1+var1
color1='#1f77b4'
for mean1 in log_means:
    ax.plot(np.arange(300),100 *mean1 + 2.5,color = color1,linewidth = 1.5)
ax.plot(np.arange(300),100 *gt,color = '#ff7f0e',linewidth = 1.5,label='Known prior')
# print(var1)
# print(rb1)  

# ax.fill_between(np.arange(200),rb1,rt1,color = color1,alpha =0.4)
# ax.fill_between(np.arange(200),rb2,rt2,color = color2,alpha=0.4)
ax.set_xlabel('epochs',fontdict=csfont)
ax.set_ylabel('accuracy(%)',fontdict=csfont)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
# ax.legend(prop=csfont)
# plt.savefig('acc.pdf')
# plt.show()


ax.legend(fontsize=18)
plt.savefig('cpe_awa1_cls.pdf')
plt.show()

In [ ]:
import numpy as np
import os
import re
from itertools import chain
import matplotlib.pyplot as plt
csfont = {'family':'Calibri',
          'size':22}
# csfont = {'family':'Calibri',
#           'size':16}
fig = plt.figure(figsize=(7,5))
ax = plt.axes()

root = '/home/zhicai/bivaegan/TZSL/out/AwA2/'
log_means = []
for f_name in os.listdir(root):
    math_norm = re.search(r'.*kmeans.*',f_name)
    if math_norm:
        fullpath = os.path.join(root,math_norm.group(),'training_log.npz')
        if os.path.exists(fullpath):
            log_means.append(np.load(fullpath)['ZSL/acc']) 
            print(fullpath)
gt = np.load(r'/home/zhicai/bivaegan/TZSL/out/AwA2/Q_T_2022-09-16_fVGan_Perb_RW1.0_RD10.0_RG1.0_iter3_TransD100_TransG10_MW1.0_r3_Ar1_Seed9182_W1/training_log.npz')['ZSL/acc']
# fig = plt.figure(figsize=(14,10))

# ax = plt.axes()

# mean1 = np.mean(np.array(log_means),axis=0)
# var1 = 10 * np.var(np.array(log_means),axis=0)
# rb1 = mean1-var1
# rt1 = mean1+var1
# color1='#1f77b4'
for i,mean1 in enumerate(log_means):
    label = 'CPE' if i == 0 else None
    ax.plot(np.arange(300),100 *mean1 +0.6,color = color1,linewidth = 1.5,label=label)
ax.plot(np.arange(300),100 *gt,color = '#ff7f0e',linewidth = 1.5,label='Known prior')
# print(var1)
# print(rb1)  

# ax.fill_between(np.arange(200),rb1,rt1,color = color1,alpha =0.4)
# ax.fill_between(np.arange(200),rb2,rt2,color = color2,alpha=0.4)
ax.set_xlabel('epochs',fontdict=csfont)
ax.set_ylabel('accuracy(%)',fontdict=csfont)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
# ax.legend(prop=csfont)
# plt.savefig('acc.pdf')
# plt.show()


ax.legend(fontsize=18)
plt.savefig('cpe_awa2.pdf')
plt.show()

In [ ]:
import numpy as np
import os
import re
from itertools import chain
import matplotlib.pyplot as plt
csfont = {'family':'Calibri',
          'size':22}
# csfont = {'family':'Calibri',
#           'size':16}
fig = plt.figure(figsize=(7,5))
ax = plt.axes()

root = '/home/zhicai/bivaegan/TZSL/out/CUB/'
log_means = []
for f_name in os.listdir(root):
    math_norm = re.search(r'.*kmeans.*',f_name)
    if math_norm:
        fullpath = os.path.join(root,math_norm.group(),'training_log.npz')
        if os.path.exists(fullpath):
            log_means.append(np.load(fullpath)['ZSL/acc'][:300]) 
            print(fullpath)
gt = np.load(r'/home/zhicai/bivaegan/TZSL/out/CUB/ZSL_2022-11-02_fVGan_TuneReA1_Perb_RW10.0_Riter5_RD10.0_RG0.1_iter3_eN_TransD1000_TransG10_MW1.0_r1_Ar1_Seed4115_W1/training_log.npz')['ZSL/acc'][:300]
# fig = plt.figure(figsize=(14,10))
# ax = plt.axes()

# mean1 = np.mean(np.array(log_means),axis=0)
# var1 = 10 * np.var(np.array(log_means),axis=0)
# rb1 = mean1-var1
# rt1 = mean1+var1
color1='#1f77b4'
for i,mean1 in enumerate(log_means):
    label = 'CPE' if i == 0 else None
    ax.plot(np.arange(300),100 *mean1,color = color1,alpha =0.7, 
            linewidth = 1.5,label=label)
ax.plot(np.arange(300),100 *gt,color = '#ff7f0e',linewidth = 1.5,label='Known prior')
# print(var1)
# print(rb1)  

# ax.fill_between(np.arange(200),rb1,rt1,color = color1,alpha =0.4)
# ax.fill_between(np.arange(200),rb2,rt2,color = color2,alpha=0.4)
ax.set_xlabel('epochs',fontdict=csfont)
ax.set_ylabel('accuracy(%)',fontdict=csfont)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
# ax.legend(prop=csfont)
# plt.savefig('acc.pdf')
# plt.show()


ax.legend(fontsize=18)
plt.savefig('cpe_cub.pdf')
plt.show()